https://huggingface.co/panggi/t5-base-indonesian-summarization-cased

- Indonesian T5 Summarization Base Model
- Finetuned T5 base summarization model for Indonesian.

Finetuning Corpus
- t5-base-indonesian-summarization-cased model is based on t5-base-bahasa-summarization-cased by huseinzol05, finetuned using indosum dataset.

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install evaluate rouge_score transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f3016b021cd0b27028f657098e46a21c70327bf0886ab708cb63f5e3dc2aa784
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    U

# Model

In [ ]:
from transformers import BertTokenizer, GPT2Tokenizer, EncoderDecoderModel
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, EarlyStoppingCallback
from datasets import Dataset, DatasetDict, load_metric
import evaluate
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np
import pandas as pd

In [ ]:
# # max_split_size_mb nya bisa dikecilin klo GPU nya OOM
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
parent_folder = "/content/drive/MyDrive/Project - 2/clean_data/" # Pake yang ada di folder "clean_data" di Google drive
train_set = pd.read_csv(parent_folder+'final_train_set.csv')
valid_set = pd.read_csv(parent_folder+'final_valid_set.csv')
test_set = pd.read_csv(parent_folder+'final_test_set.csv')

Mounted at /content/drive


In [ ]:
train_set = train_set.sample(n=2000, random_state=42).reset_index(drop=True)
valid_set = valid_set.sample(n=1000, random_state=42).reset_index(drop=True)
test_set = test_set.sample(n=1000, random_state=42).reset_index(drop=True)

In [ ]:
train_set.shape, valid_set.shape, test_set.shape

((2000, 2), (1000, 2), (1000, 2))

In [ ]:
train_set.duplicated().sum(), valid_set.duplicated().sum(), test_set.duplicated().sum()

(0, 0, 0)

In [ ]:
train_set.head()

,final_clean_article,combined_clean_summary
0,"Suasana aksi unjuk rasa di depan Gedung DPR, J...","Aksi demo di depan gedung DPR, Senayan Jakarta..."
1,KABUT duka menyelimuti presenter dan bintang s...,KABUT duka menyelimuti presenter dan bintang s...
2,Lokasi wisata Taman Ade Irma Suryani di Cirebo...,Letaknya yang dekat dengan Pelabuhan Cirebon m...
3,TAK ada yang janggal dari penampilan duo RATU ...,TAK ada yang janggal dari penampilan duo RATU ...
4,Keganasan Liverpool kala menggulung dua tim ra...,"Winger Liverpool asal Spanyol, Albert Riera, m..."


In [ ]:
valid_set.head()

,final_clean_article,combined_clean_summary
0,Satuan Tugas Timor yang dibentuk sejak pekan s...,Restrukturisasi PT Timor Putra Nasional dihara...
1,"Kondisi Taman Nasional Gunung Leuser, Medan, S...",Seluas 9. 000 hektare lahan di Taman Nasional ...
2,Antraks bukan penyakit yang aneh. Penyakit ter...,Kepala Dinas Peternakan DKI Edy Setiarto meneg...
3,Wakil Presiden Megawati Sukarnoputri berharap ...,Wapres Megawati Sukarnoputri mencanangkan akse...
4,Pemerintah Daerah Yogyakarta menolak pengurang...,"Besar kemungkinan, dana alokasi umum untuk dae..."


In [ ]:
train_dataset = Dataset.from_pandas(train_set)
valid_dataset = Dataset.from_pandas(valid_set)
test_dataset = Dataset.from_pandas(test_set)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'val': valid_dataset,
    'test': test_dataset
})

## Define Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["final_clean_article"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    labels = tokenizer(text_target=examples["combined_clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="rouge1",  # Metric to use for early stopping
    greater_is_better=True,  # Whether a higher value of the metric is better
    save_steps=500,  # Define the number of steps after which to save the model
    eval_steps=500,  # Define the number of steps after which to run evaluation
)

In [ ]:
rouge = evaluate.load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    ### BOLEH DIGANTI SESUAI KEBUTUHAN MODEL ###
    # if isinstance(predictions, tuple):
    #     predictions = predictions[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    ### ###

    # Directly decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Rouge expects newline-separated text
    decoded_preds = ["\n".join(decoded_pred.split()) for decoded_pred in decoded_preds]
    decoded_labels = ["\n".join(decoded_label.split()) for decoded_label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    compute_metrics=compute_metrics,
    data_collator=data_collator
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.891500,2.513030,0.388600,0.235300,0.348300,0.388800
2,0.550000,2.510119,0.381500,0.231900,0.343000,0.382100
3,0.378100,2.575534,0.395300,0.245200,0.356200,0.395400
4,0.251600,2.625597,0.389300,0.238500,0.349500,0.389700
5,0.186000,2.674479,0.392200,0.242000,0.352900,0.392200
6,0.143200,2.701040,0.383300,0.233500,0.343600,0.383400
7,0.114400,2.736818,0.395100,0.245600,0.355500,0.395500
8,0.085000,2.749164,0.402900,0.252800,0.364200,0.402900
9,0.075600,2.775384,0.400400,0.250900,0.361300,0.400500
10,0.066000,2.780780,0.398100,0.247800,0.359300,0.398200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Some

TrainOutput(global_step=630, training_loss=0.276658972199001, metrics={'train_runtime': 1985.8104, 'train_samples_per_second': 10.071, 'train_steps_per_second': 0.317, 'total_flos': 6134586408960000.0, 'train_loss': 0.276658972199001, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print(eval_results)

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 2.461379051208496, 'eval_rouge1': 0.4001, 'eval_rouge2': 0.2584, 'eval_rougeL': 0.3602, 'eval_rougeLsum': 0.4, 'eval_runtime': 159.0273, 'eval_samples_per_second': 6.288, 'eval_steps_per_second': 0.201, 'epoch': 10.0}


In [ ]:
# Save the model
model.save_pretrained("bert2bert_cahya")

# Save the tokenizer
tokenizer.save_pretrained("bert2bert_cahya/tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


('bert2bert_cahya/tokenizer/tokenizer_config.json',
 'bert2bert_cahya/tokenizer/special_tokens_map.json',
 'bert2bert_cahya/tokenizer/vocab.txt',
 'bert2bert_cahya/tokenizer/added_tokens.json')

In [ ]:
# # Load from local
model = EncoderDecoderModel.from_pretrained("bert2bert_cahya")

# # Load tokenizer from local
# tokenizer = BertTokenizer.from_pretrained("nama_model/tokenizer")

In [ ]:
%%time
ARTICLE_TO_SUMMARIZE = """Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC. Fungsi dari Kartu Debit Mandiri ini sangat beragam, mulai dari tarik tunai, setor tunai, transfer uang, cek saldo rekening, hingga membayar berbagai tagihan melalui mesin ATM.
Penting bagi Anda yang ingin membuka rekening tabungan di Bank Mandiri untuk memahami jenis Kartu Debit Mandiri agar tidak salah memilih. Setiap kartu debit Mandiri memiliki kelebihan dan kekurangannya masing-masing, sehingga penting bagi nasabah untuk memilih yang sesuai dengan kebutuhan dan preferensi mereka.

Dalam memilih jenis Kartu Debit Mandiri, nasabah perlu mempertimbangkan kebutuhan dan gaya hidup mereka. Apakah mereka membutuhkan manfaat tambahan seperti asuransi atau akses ke airport lounge, ataukah mereka menginginkan kartu debit yang sederhana namun praktis. Dengan mengetahui jenis Kartu Debit Mandiri yang sesuai, nasabah dapat memaksimalkan manfaat yang mereka dapatkan dari penggunaan kartu debit tersebut.
"""

# generate summary
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt')
summary_ids = model.generate(input_ids.to(model.device),
            min_length=20,
            max_length=128,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)
# start time dan end time
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

kartu debit mandiri dapat digunakan oleh nasabahnya untuk melakukan berbagai transaksi di mesin atm atau mesin edc.
CPU times: user 16.3 s, sys: 793 ms, total: 17.1 s
Wall time: 2.97 s
